# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [40]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
print(f"module_path: {module_path}")
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

module_path: /home/beegass/Documents/Coding/s4mer


In [41]:
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "False"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"

In [42]:
## import packages
import math
from typing import Any, Callable, List, Optional, Tuple, Union

import jax
import jax.numpy as jnp

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import requests
from flax import linen as jnn
from jax.nn.initializers import lecun_normal, uniform
from jax.numpy.linalg import eig, inv, matrix_power
from jax.scipy.signal import convolve
from jaxtyping import Array, Float, Float16, Float32, Float64
from scipy import linalg as la
from scipy import signal
from scipy import special as ss

from src.data.process import moving_window, rolling_window
from src.models.hippo.gu_hippo import gu_HiPPO_LSI, gu_HiPPO_LTI

# import modules
from src.models.hippo.gu_transition import GuTransMatrix
from src.models.hippo.hippo import HiPPOLSI, HiPPOLTI
from src.models.hippo.transition import TransMatrix
from src.models.hippo.unroll import (
    basis,
    measure,
    variable_unroll_matrix,
    variable_unroll_matrix_sequential,
)
from src.utils.ops import genlaguerre, legendre_polynomial

print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [43]:
from functools import partial
from typing import Any

import einops
import functorch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [44]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [45]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [46]:
num_copies = 10
subkeys = jax.random.split(key, num=num_copies)
key = subkeys[0]

In [47]:
def whitesignal(period, dt, freq, rms=0.5, batch_shape=()):
    """
    Produces output signal of length period / dt, band-limited to frequency freq
    Output shape (*batch_shape, period/dt)
    Adapted from the nengo library
    """

    if freq is not None and freq < 1.0 / period:
        raise ValueError(
            f"Make ``{freq=} >= 1. / {period=}`` to produce a non-zero signal",
        )

    nyquist_cutoff = 0.5 / dt
    if freq > nyquist_cutoff:
        raise ValueError(
            f"{freq} must not exceed the Nyquist frequency for the given dt ({nyquist_cutoff:0.3f})"
        )

    n_coefficients = int(np.ceil(period / dt / 2.0))
    shape = batch_shape + (n_coefficients + 1,)
    sigma = rms * np.sqrt(0.5)
    coefficients = 1j * np.random.normal(0.0, sigma, size=shape)
    coefficients[..., -1] = 0.0
    coefficients += np.random.normal(0.0, sigma, size=shape)
    coefficients[..., 0] = 0.0

    set_to_zero = np.fft.rfftfreq(2 * n_coefficients, d=dt) > freq
    coefficients *= 1 - set_to_zero
    power_correction = np.sqrt(1.0 - np.sum(set_to_zero, dtype=float) / n_coefficients)
    if power_correction > 0.0:
        coefficients /= power_correction
    coefficients *= np.sqrt(2 * n_coefficients)
    signal = np.fft.irfft(coefficients, axis=-1)
    signal = signal - signal[..., :1]  # Start from 0
    return signal

## Test HiPPO Reconstruction

In [48]:
def test_hippo_reconstruction(
    hippo, gu_hippo, random_input, key, s_or_t="lti", print_all=False
):
    x_tensor = torch.tensor(random_input, dtype=torch.float32)
    x_jnp = jnp.asarray(x_tensor, dtype=jnp.float32)  # convert torch array to jax array

    # My Implementation
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"----------------------------My {s_or_t} Implementation Outputs----------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_jnp = einops.rearrange(x_jnp, "batch seq_len -> batch seq_len 1")
    params = hippo.init(key, f=x_jnp)
    hippo = hippo.bind(params)
    c = hippo.__call__(f=x_jnp)
    print(f"c shape: {c.shape}")
    y = hippo.reconstruct(c)
    print(f"y shape: {y.shape}")

    # Gu's HiPPO LegS
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Gu's {s_or_t} Implementation Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_tensor = torch.moveaxis(x_tensor, 0, 1)
    GU_c_s, GU_c_k = gu_hippo(x_tensor, fast=False)
    print(f"c_s shape: {GU_c_s.shape}")
    print(f"c_k shape: {GU_c_k.shape}")

    # c shape: (1, 1, 64)
    # y shape: (1, 1, 64)
    # c_s shape: torch.Size([3000, 1, 64])
    # c_k shape: torch.Size([1, 64])

    gu_y = None
    if s_or_t == "lsi":
        GU_c_s = einops.rearrange(GU_c_s, "seq_len batch N -> batch seq_len 1 N")
        gu_y = gu_hippo.reconstruct(GU_c_s)
    elif s_or_t == "lti":
        GU_c_k = einops.rearrange(GU_c_k, "batch N -> batch 1 N")
        gu_y = gu_hippo.reconstruct(GU_c_k)
    else:
        raise ValueError(
            f"s_or_t must be either 'lsi' or 'lti'. s_or_t is currently set to: {s_or_t}"
        )

    if s_or_t == "lsi":
        gu_c = jnp.asarray(
            GU_c_s, dtype=jnp.float32
        )  # convert torch array to jax array
        # (1, 10000, 10000, 1)
        print("LSI:\n")
        print(f"gu_c shape: {gu_c.shape}")
        print(f"gu_y shape: {gu_y.shape}")
        print(f"y shape: {y.shape}")
        y = einops.rearrange(
            y, "batch seq_len seq_len2 input_len -> batch seq_len input_len seq_len2"
        )
        gu_y = einops.rearrange(
            gu_y, "seq_len batch seq_len2 input_len -> batch seq_len input_len seq_len2"
        )
        print(f"gu_y shape: {gu_y.shape}")
        print(f"y shape: {y.shape}")

    elif s_or_t == "lti":
        gu_c = jnp.asarray(
            GU_c_k, dtype=jnp.float32
        )  # convert torch array to jax array
        # (1, 64, 1)
        print("LTI:\n")
        print(f"gu_c shape: {gu_c.shape}")
        print(f"gu_y shape: {gu_y.shape}")
        print(f"y shape: {y.shape}")
        y = einops.rearrange(
            y, "batch seq_len input_len -> batch input_len seq_len"
        )  # transposing
        gu_y = einops.rearrange(
            gu_y, "batch seq_len input_len -> batch input_len seq_len"
        )  # transposing
        print(f"gu_y shape: {gu_y.shape}")
        print(f"y shape: {y.shape}")

    gu_y = jnp.asarray(gu_y, dtype=jnp.float32)  # convert torch array to jax array

    x_tensor = jnp.asarray(
        x_tensor, dtype=jnp.float32
    )  # convert torch array to jax array
    x_tensor = jnp.moveaxis(x_tensor, 0, 1)

    co_flag = True
    y_flag = True
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):

            co_check = None
            y_check = None
            if s_or_t == "lsi":
                co_check = jnp.allclose(
                    c[i, j, :, :], gu_c[i, j, :, :], rtol=1e-03, atol=1e-03
                )
                y_check = jnp.allclose(
                    y[i, j, :, :], gu_y[i, j, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"c_k[{i},{j},:,:]:\n{c[i,j,:,:]}")
                    print(f"gu_cs[{i},{j},:,:]:\n{gu_c[i,j,:,:]}")
                    print(f"coefficients check: {co_check}\n")

                    print(f"y[{i},{j},:,:]:\n{y[i,j,:,:]}")
                    print(f"gu_y[{i},{j},:,:]:\n{gu_y[i,j,:,:]}")
                    print(f"reconstruction check: {y_check}\n")

            elif s_or_t == "lti":
                co_check = jnp.allclose(
                    c[i, j, :], gu_c[i, :, :], rtol=1e-03, atol=1e-03
                )
                y_check = jnp.allclose(
                    y[i, j, :], gu_y[i, :, :], rtol=1e-03, atol=1e-03
                )

                if print_all:
                    print(f"c_k[{i},{j},:]:\n{c[i,:,:]}")
                    print(f"gu_cs[{i},{j},:]:\n{gu_c[i,j,:]}")
                    print(f"coefficients check: {co_check}\n")

                    print(f"y[{i},{j},:]:\n{y[i,:,:]}")
                    print(f"gu_y[{i},{j},:]:\n{gu_y[i,:,:]}")
                    print(f"reconstruction check: {y_check}\n")

            if co_check == False:
                co_flag = False

            if y_check == False:
                y_flag = False

    if not print_all:
        print(f"\n------------------------------------------------------------")
        print(f"---------- The Coefficients Test Passed: {co_flag} ----------")
        print(f"-------------------------------------------------------------\n\n")
        print(f"\n-------------------------------------------------------------")
        print(f"---------- The Reconstruction Test Passed: {y_flag} ----------")
        print(f"--------------------------------------------------------------\n")

    if print_all:
        print(f"c:\n {c}")
        print(f"c shape:\n {c.shape}")

        print(f"gu_c:\n {gu_c}")
        print(f"gu_c shape:\n {gu_c.shape}")

        print(f"y:\n {y}")
        print(f"y shape:\n {y.shape}")

        print(f"gu_y:\n {gu_y}")
        print(f"gu_y shape:\n {gu_y.shape}")

In [49]:
def test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    # N = 256
    # L = 128

    T = 3
    freq = 3
    step = 1e-3
    L = int(T / step)

    batch_size = 2
    data_size = L
    input_size = 1

    N = 64

    u = whitesignal(T, step, freq, batch_shape=(batch_size,))

    x_np = np.asarray(u)

    x = torch.tensor(x_np, dtype=torch.float32)

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = gu_HiPPO_LTI(
        N=N,
        method=the_measure,
        dt=step,
        T=T,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = gu_HiPPO_LSI(
            N=N,
            method=the_measure,
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate My HiPPOs -----------------------------
    # ----------------------------------------------------------------------------------
    print(f"\nTesting BRYANS HiPPO-{the_measure} model")

    matrices = TransMatrix(
        N=N,
        measure=the_measure,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        dtype=jnp.float32,
    )

    A = matrices.A
    B = matrices.B

    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPOLTI(
        N=N,
        step_size=step,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        basis_size=T,
        dtype=jnp.float32,
        unroll=False,
    )  # Bryan's

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPOLSI(
            N=N,
            max_length=L,
            step_size=step,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            dtype=jnp.float32,
            unroll=True,
        )  # Bryan's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Test HiPPO Operators ------------------------------
    # ----------------------------------------------------------------------------------

    print(f"Bryan's Coeffiecients for {alpha} LTI HiPPO-{the_measure}")

    test_hippo_reconstruction(
        hippo=hippo_lti,
        gu_hippo=gu_hippo_lti,
        random_input=x_np,
        key=subkeys[5],
        s_or_t="lti",
        print_all=print_all,
    )

    if the_measure == "legs":
        print(f"\n\nBryan's Coeffiecients for {alpha} LSI HiPPO-{the_measure}")

        test_hippo_reconstruction(
            hippo=hippo_lsi,
            gu_hippo=gu_hippo_lsi,
            random_input=x_np,
            key=subkeys[6],
            s_or_t="lsi",
            print_all=print_all,
        )

    print(f"end of test for HiPPO-{the_measure} model")

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [50]:
print_all = False

### Testing (LTI and LSI) Operators With Forward Euler Transform

#### LegS

In [51]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legs
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------



Bryan's Coeffiecients for 0.0 LSI HiPPO-legs
c shape: (2, 3000, 1, 64)
y shape: (2, 30

#### LegT

In [52]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [92]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lmu
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [91]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lagt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FRU

In [90]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fru
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fru model


#### FouT

In [89]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fout
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


#### FouD

In [88]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-foud
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Backward Euler Transform

#### LegS

In [58]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legs
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------



Bryan's Coeffiecients for 1.0 LSI HiPPO-legs
c shape: (2, 3000, 1, 64)
y shape: (2, 30

#### LegT

In [87]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 1.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [86]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 1.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lmu
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [85]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 1.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lagt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FRU

In [84]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 1.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fru
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fru model


#### FouT

In [83]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 1.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fout
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


#### FouD

In [82]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 1.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-foud
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With Bidirectional Transform

#### LegS

In [65]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legs
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------



Bryan's Coeffiecients for 0.5 LSI HiPPO-legs
c shape: (2, 3000, 1, 64)
y shape: (2, 30

#### LegT

In [66]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.5 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [67]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.5 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lmu
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [68]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.5 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lagt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FRU

In [69]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.5 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fru
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fru model


#### FouT

In [70]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.5 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fout
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


#### FouD

In [71]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.5 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-foud
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: False ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: False ----------
--------------------------------------------------------------

end of test for HiPPO-foud model


### Testing (LTI and LSI) Operators With ZOH Transform

#### LegS

In [72]:
test_reconstruction(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legs
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------



Bryan's Coeffiecients for 2.0 LSI HiPPO-legs
c shape: (2, 3000, 1, 64)
y shape: (2, 3000

#### LegT

In [73]:
test_reconstruction(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legt LTI model with 2.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-legt model


#### LMU

In [74]:
test_reconstruction(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lmu LTI model with 2.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lmu
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lmu model


#### LagT

In [81]:
test_reconstruction(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lagt LTI model with 2.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lagt
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-lagt model


#### FRU

In [80]:
test_reconstruction(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fru LTI model with 2.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fru
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-fru model


#### FouT

In [79]:
test_reconstruction(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fout LTI model with 2.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fout
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-fout model


#### FouD

In [78]:
test_reconstruction(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-foud LTI model with 2.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-foud
c shape: (2, 1, 64)
y shape: (2, 3000, 1)
c_s shape: torch.Size([3000, 2, 64])
c_k shape: torch.Size([2, 64])
LTI:

gu_c shape: (2, 1, 64)
gu_y shape: torch.Size([2, 3000, 1])
y shape: (2, 3000, 1)
gu_y shape: torch.Size([2, 1, 3000])
y shape: (2, 1, 3000)

------------------------------------------------------------
---------- The Coefficients Test Passed: True ----------
-------------------------------------------------------------



-------------------------------------------------------------
---------- The Reconstruction Test Passed: True ----------
--------------------------------------------------------------

end of test for HiPPO-foud model
